In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install transformers

     |████████████████████████████████| 3.8 MB 8.6 MB/s 
     |████████████████████████████████| 596 kB 56.6 MB/s 
     |████████████████████████████████| 895 kB 62.7 MB/s 
     |████████████████████████████████| 67 kB 7.9 MB/s 
     |████████████████████████████████| 6.5 MB 52.8 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [3]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
import torch.nn as nn
import torch

import transformers
from transformers import AdamW, get_linear_schedule_with_warmup
import os

### Data Process

In [4]:
for dirpath, dirnames, filenames in os.walk('./drive/MyDrive/Document_Classification/data/text/'):
    idx2text = {}
    for filename in filenames:
        idx = filename.split('.')[0]
        with open(dirpath + filename) as f:
            abstract = ''
            for sentence in f:
                abstract += sentence
            abstract = abstract.replace('\n', ' ')
            idx2text[idx] = abstract

for dirpath, dirnames, filenames in os.walk('./drive/MyDrive/Document_Classification/data/labels/'):
    idx2label = []
    for filename in filenames:
        idx = filename.split('.')[0]
        with open(dirpath + filename) as f:
            labels = f.readline()
            for label in labels.split('<'):
                if not label or label.isspace():
                    continue
                key_label = label.split('--')[0].strip()
                if key_label == 'NULL':
                    continue
                idx2label.append((idx, key_label))

In [5]:
idx2label = list(set(idx2label))
df = pd.DataFrame(idx2label, columns=['idx', 'label'])
df['text'] = df['idx'].map(idx2text)

In [6]:
label_vals = list(set(df['label'].tolist()))

In [7]:
df = df.groupby(['idx', 'text']).agg({'label': lambda x: list(x)}).reset_index()

In [8]:
mlb = MultiLabelBinarizer()
df = df.join(pd.DataFrame(mlb.fit_transform(df.pop('label')),
                          columns=mlb.classes_,
                          index=df.index))

In [9]:
df.head()

,idx,text,Activating invasion and metastasis,Avoiding immune destruction,Cellular energetics,Enabling replicative immortality,Evading growth suppressors,Genomic instability and mutation,Inducing angiogenesis,Resisting cell death,Sustaining proliferative signaling,Tumor promoting inflammation
0,11724768,Ghrelin was identified in the stomach as an en...,0,0,0,0,0,0,0,0,1,0
1,11773160,PURPOSE The epidermal growth factor receptor (...,0,0,0,0,1,0,0,0,1,0
2,11774243,Adoptive transfer of immunity against hepatiti...,0,1,0,0,0,0,0,0,0,0
3,11781072,The secretion of immunosuppressive factors lik...,0,1,0,0,0,0,0,1,1,0
4,11791181,To characterize the impact of increased produc...,1,0,0,0,0,0,0,0,1,0


In [10]:
del idx2label, idx2text

### Model

In [11]:
class HoCDataset:
    def __init__(self, tokenizer, sentences, labels, max_len):
        self.sentences = sentences
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.sentences)

    def __getitem__(self, item):
        sentence = str(self.sentences[item])
        inputs = self.tokenizer.encode_plus(
            sentence,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            pad_to_max_length=True,
        )
        ids = inputs['input_ids']
        token_type_ids = inputs['token_type_ids']
        mask = inputs['attention_mask']

        return {
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long),
            'token_type_ids': torch.tensor(token_type_ids, dtype=torch.long),
            'label': torch.tensor(self.labels[item], dtype=torch.float)
        }


class BERTClass(nn.Module):
    def __init__(self):
        super(BERTClass, self).__init__()
        self.bert = transformers.BertModel.from_pretrained('bert-base-uncased')
        self.out = nn.Linear(768, 10)


    def forward(self, ids, mask, token_type_ids):
        _, output = self.bert(ids, attention_mask=mask, token_type_ids=token_type_ids, return_dict=False)

        return self.out(output)


def loss_fn(outputs, targets):
    return nn.BCEWithLogitsLoss()(outputs, targets)


def train_loop_fn(data_loader, model, optimizer, device, scheduler=None):
    model.train()
    for bi, d in enumerate(data_loader):
        ids = d['ids']
        mask = d['mask']
        token_type_ids = d['token_type_ids']
        labels = d['label']

        ids = ids.to(device, dtype=torch.long)
        mask = mask.to(device, dtype=torch.long)
        token_type_ids = token_type_ids.to(device, dtype=torch.long)
        labels = labels.to(device, dtype=torch.float)

        optimizer.zero_grad()
        outputs = model(ids, mask, token_type_ids)

        loss = loss_fn(outputs, labels)
        loss.backward()
        optimizer.step()
        if scheduler is not None:
            scheduler.step()
        if bi % 50 == 0:
            print(f'bi={bi}, loss={loss}')


def eval_loop_fn(data_loader, model, device):
    model.eval()
    fin_labels = []
    fin_outputs = []
    for bi, d in enumerate(data_loader):
        with torch.no_grad():
            ids = d['ids'].to(device, dtype=torch.long)
            mask = d['mask'].to(device, dtype=torch.long)
            token_type_ids = d['token_type_ids'].to(device, dtype=torch.long)
            labels = d['label'].to(device, dtype=torch.long)

            outputs = model(ids, mask, token_type_ids)

            fin_labels.append(labels.cpu().detach().numpy())
            fin_outputs.append(torch.sigmoid(outputs).cpu().detach().numpy())

    return np.vstack(fin_outputs), np.vstack(fin_labels)

In [12]:
MAX_LEN = 512
TRAIN_BATCH_SIZE = 8
EPOCHS = 4
SEED = 42
LEARNING_RATE = 3e-5
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


label_cols = list(df.drop(['idx', 'text'], axis=1).columns)
labels = df[label_cols].values

df_train, df_test, labels_train, labels_test = train_test_split(
    df, labels, test_size=0.2, random_state=SEED)


tokenizer = transformers.BertTokenizer.from_pretrained('bert-base-uncased')
model = BERTClass().to(device)

train_dataset = HoCDataset(
    sentences=df_train.text.values,
    labels=labels_train,
    tokenizer=tokenizer,
    max_len=MAX_LEN
)
train_data_loader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=TRAIN_BATCH_SIZE,
    shuffle=True
)

test_dataset = HoCDataset(
    sentences=df_test.text.values,
    labels=labels_test,
    tokenizer=tokenizer,
    max_len=MAX_LEN
)
test_data_loader = torch.utils.data.DataLoader(
    test_dataset,
    batch_size=4,
    shuffle=True,
    drop_last=True
)

optimizer = AdamW(model.parameters(), lr=LEARNING_RATE)
num_training_steps = int(len(df_train) / TRAIN_BATCH_SIZE * EPOCHS)
scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps
)

for epoch in range(EPOCHS):
    train_loop_fn(train_data_loader, model, optimizer, device, scheduler)
output, target = eval_loop_fn(test_data_loader, model, device)

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/420M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a

bi=0, loss=0.7137302756309509
bi=50, loss=0.438169926404953
bi=100, loss=0.40185537934303284
bi=150, loss=0.3818751871585846
bi=0, loss=0.31327879428863525
bi=50, loss=0.1991667002439499
bi=100, loss=0.3344514071941376
bi=150, loss=0.22547197341918945
bi=0, loss=0.22841401398181915
bi=50, loss=0.1866484433412552
bi=100, loss=0.22636234760284424
bi=150, loss=0.19424138963222504
bi=0, loss=0.18188688158988953
bi=50, loss=0.16156414151191711
bi=100, loss=0.14934010803699493
bi=150, loss=0.14779141545295715


In [13]:
from sklearn import metrics
output = np.array(output) >= 0.5
f1_score_micro = metrics.f1_score(target, output, average='micro')
print(f"F1 Score (Micro) = {f1_score_micro}")

F1 Score (Micro) = 0.803088803088803
